In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import wfdb
# from collections import Counter
# from sklearn.preprocessing import LabelEncoder
# from imblearn.over_sampling import SMOTENC
# from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

In [ ]:
X1 = np.load('Atrial_Fibrillation_ECG_signal_1780_patients_X.npy')
y_data_1 = np.load('Atrial_Fibrillation_ECG_signal_1780_patients_y_data_1.npy')
X2 = np.load('NORMAL_ECG_signal_5000_patients_X.npy')
y_data_0 = np.load('NORMAL_ECG_signal_5000_patients_y_data_0.npy')

In [ ]:
X = np.concatenate((X1,X2),axis=0)
y_data = np.concatenate((y_data_1,y_data_0),axis=0)

In [ ]:
y_data.shape

In [ ]:
# X, y_data = shuffle(X, y_data, random_state=42)

In [ ]:
xnew = []
ynew = []
patient_ids = []
window_size = 1000  

for i in range(X.shape[0]):
    for j in range(5):  
        start = j * window_size
        end = (j + 1) * window_size
        xnew.append(X[i, :, start:end])
        ynew.append(y_data[i]) 
        patient_ids.append(i)

xnew = np.array(xnew)
ynew = np.array(ynew)
patient_ids = np.array(patient_ids)

In [ ]:
del X1
del X2
del X
del y_data_1
del y_data_0

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV

In [ ]:
rocket=Rocket()
# rocket=Rocket(num_kernels=10000)
rocket.fit(xnew)
xt=rocket.transform(xnew)

In [ ]:
# np.save('xt_1798+5000_windowed5_10000_kernels_not_random.npy', xt)
# np.save('ynew_1798+5000_windowed5_10000_kernels_not_random.npy', ynew)
# np.save('patient_ids_1798+5000_windowed5_10000_kernels_not_random.npy', patient_ids)

In [ ]:
xt2 = xt
ynew2 = ynew
patient_ids2 = patient_ids

In [ ]:
# xt = np.load('xt_1798+5000_windowed5_7000_kernels_not_random.npy')
# ynew = np.load('ynew_1798+5000_windowed5_7000_kernels_not_random.npy')
# patient_ids = np.load('patient_ids_1798+5000_windowed5_7000_kernels_not_random.npy')

In [ ]:
xt, ynew, patient_ids = shuffle(xt, ynew, patient_ids, random_state=42)

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0
sum_f1 = 0

for train_idx, test_idx in gkf.split(xt, ynew, groups=patient_ids):
    X_train, X_test = xt.iloc[train_idx], xt.iloc[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    classifier = SGDClassifier()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


In [ ]:
# window = 5
# unhealthy = 1780
# folds = 4

# unhealthy_x = xt2[:unhealthy*window]
# healthy_x = xt2[unhealthy*window:]
# unhealthy_y = ynew2[:unhealthy*window]
# healthy_y = ynew2[unhealthy*window:]

# from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
# from sklearn.model_selection import GroupKFold
# from sklearn.linear_model import SGDClassifier
# import numpy as np

# gkf1 = GroupKFold(n_splits=folds)
# gkf2 = GroupKFold(n_splits=folds)

# sum_accuracy = 0
# sum_f1 = 0

# for train_idx1, test_idx1 in gkf1.split(xt2[:unhealthy*window], ynew2[:unhealthy*window], groups=patient_ids2[:unhealthy*window]):
#     sum_accuracy = 0
#     X_train1, X_test1 = xt2.iloc[train_idx1], xt2.iloc[test_idx1]
#     y_train1, y_test1 = ynew2[train_idx1], ynew2[test_idx1]

#     for train_idx2, test_idx2 in gkf2.split(xt2[unhealthy*window:], ynew2[unhealthy*window:], groups=patient_ids2[unhealthy*window:]):
#         X_train2, X_test2 = healthy_x.iloc[train_idx2], healthy_x.iloc[test_idx2]
#         y_train2, y_test2 = healthy_y[train_idx2], healthy_y[test_idx2]

#         X_train = np.concatenate((X_train1, X_train2), axis=0) 
#         X_test = np.concatenate((X_test1, X_test2), axis=0) 

#         y_train = np.concatenate((y_train1, y_train2), axis=0) 
#         y_test = np.concatenate((y_test1, y_test2), axis=0)
        
#         classifier = SGDClassifier()
#         classifier.fit(X_train, y_train)

#         y_pred = classifier.predict(X_test)
        
#         y_pred_majority = []
#         for i in range(0, y_pred.shape[0], window):
#             majority_vote = np.bincount(y_pred[i:i+window]).argmax()
#             y_pred_majority.append(majority_vote)

#         y_test_majority = []
#         for i in range(0, y_test.shape[0], window):
#             majority_vote = np.bincount(y_test[i:i+window]).argmax()
#             y_test_majority.append(majority_vote)

#         accuracy = accuracy_score(y_test_majority, y_pred_majority)
#         sum_accuracy += accuracy

#         print(confusion_matrix(y_test_majority, y_pred_majority))
#         print("Accuracy:", accuracy)

#     average_accuracy = sum_accuracy / folds
#     print("Average Accuracy:", average_accuracy)


In [ ]:
# window = 8
# unhealthy = 460
# folds = 3

# unhealthy_x = xt[:unhealthy*window]
# healthy_x = xt[unhealthy*window:]
# unhealthy_y = ynew[:unhealthy*window]
# healthy_y = ynew[unhealthy*window:]

# from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
# from sklearn.model_selection import GroupKFold
# from sklearn.linear_model import SGDClassifier
# import numpy as np

# gkf1 = GroupKFold(n_splits=folds)
# gkf2 = GroupKFold(n_splits=folds)

# sum_accuracy = 0
# sum_sensitivity = 0
# sum_specificity = 0
# sum_f1 = 0

# # for train_idx1, test_idx1,train_idx2, test_idx2  in gkf1.split(xt[:800*5], ynew[:800*5], groups=patient_ids[:800*5]),gkf2.split(xt[800*5:], ynew[800*5:], groups=patient_ids[800*5:]):
# for train_idx1, test_idx1  in gkf1.split(xt[:unhealthy*window], ynew[:unhealthy*window], groups=patient_ids[:unhealthy*window]):
#     sum_accuracy = 0
#     X_train1, X_test1 = xt.iloc[train_idx1], xt.iloc[test_idx1]
#     y_train1, y_test1 = ynew[train_idx1], ynew[test_idx1]

#     for train_idx2, test_idx2  in gkf2.split(xt[unhealthy*window:], ynew[unhealthy*window:], groups=patient_ids[unhealthy*window:]):
#         X_train2, X_test2 = healthy_x.iloc[train_idx2], healthy_x.iloc[test_idx2]
#         y_train2, y_test2 = healthy_y[train_idx2], healthy_y[test_idx2]

#         X_train = np.concatenate((X_train1, X_train2), axis=0) 
#         X_test = np.concatenate((X_test1, X_test2), axis=0) 

#         y_train = np.concatenate((y_train1, y_train2), axis=0) 
#         y_test = np.concatenate((y_test1, y_test2), axis=0)
        
#     # for train_idx, test_idx in gkf.split(xt, ynew, groups=patient_ids):
#     #     X_train, X_test = xt.iloc[train_idx], xt.iloc[test_idx]
#     #     y_train, y_test = ynew[train_idx], ynew[test_idx]

#         classifier = SGDClassifier()
#         classifier.fit(X_train, y_train)

#         y_pred = classifier.predict(X_test)
        
#         y_pred_majority = []
#         for i in range(0, y_pred.shape[0], window):
#             majority_vote = np.bincount(y_pred[i:i+window]).argmax()
#             y_pred_majority.append(majority_vote)

#         y_test_majority = []
#         for i in range(0, y_test.shape[0], window):
#             majority_vote = np.bincount(y_test[i:i+window]).argmax()
#             y_test_majority.append(majority_vote)

#         accuracy = accuracy_score(y_test_majority, y_pred_majority)
#         sum_accuracy += accuracy

#     #     tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

#     #     sensitivity = tp / (tp + fn)
#     #     specificity = tn / (tn + fp)
#     #     f1 = f1_score(y_test_majority, y_pred_majority)

#     #     sum_sensitivity += sensitivity
#     #     sum_specificity += specificity
#     #     sum_f1 += f1

#         print(confusion_matrix(y_test_majority, y_pred_majority))
#         print("Accuracy:", accuracy)
#     #     print("Sensitivity:", sensitivity)
#     #     print("Specificity:", specificity)
#     #     print("F1-Score:", f1)
#     #     print()

#     average_accuracy = sum_accuracy / folds
#     # average_sensitivity = sum_sensitivity / 5
#     # average_specificity = sum_specificity / 5
#     # average_f1 = sum_f1 / 5

#     print("Average Accuracy:", average_accuracy)
#     # print("Average Sensitivity:", average_sensitivity)
#     # print("Average Specificity:", average_specificity)
#     # print("Average F1-Score:", average_f1)


In [ ]:
# from sklearn.linear_model import RidgeClassifierCV
# from sktime.transformations.panel.rocket import Rocket
# from sklearn.metrics import accuracy_score
# from sklearn.linear_model import SGDClassifier, RidgeClassifierCV
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import scale

# X_train, X_test, y_train, y_test = train_test_split(xt, y_data,
#                                                     test_size=0.25,
#                                                     random_state=42)

# classifier=SGDClassifier()
# classifier.fit(X_train,y_train)

# ypred=classifier.predict(X_test)

# print(confusion_matrix(y_test,ypred))
# print(accuracy_score(y_test,ypred))

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from biosppy.signals import ecg

import wfdb
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
from scipy.signal import butter, filtfilt, detrend

# record_name = files_unhealthy[0].replace("\n", "")

path = 'C:\\Users\\Pushpam\\Downloads\\'


record_name = 'JS00001'

# # Load the ECG record
# record = wfdb.rdrecord(path + record_name)

# # Extract the ECG signal
# ecg_signal = record.p_signal[:, 5] 

# # Perform baseline wander removal
# filtered_ecg = signal.detrend(ecg_signal)
# filtered_ecg=filtered_ecg[:10000]
# # Plot the ECG signal with detected R-peaks

signal, meta_val = wfdb.rdsamp(path + record_name)
value = signal.T
ecg_signals = value

filtered_ecg =X[700,3,:2000]#(detrend(ecg_signals))[2,:]

# print(filtered_ecg)
plt.figure(figsize=(10, 4))
plt.plot(np.arange(len(filtered_ecg)) / 1000, filtered_ecg, 'b')
# plt.plot(qrs_inds / record.fs, filtered_ecg[qrs_inds], 'ro', markersize=4)
plt.grid(True)
plt.show()